In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


1. Sub 1 is a prophet model, with lb of 10
2. 

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import warnings
warnings.filterwarnings("ignore")
from fbprophet import Prophet

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train = pd.read_csv('/content/drive/My Drive/JantaHack/train_aWnotuB.csv')
test = pd.read_csv('/content/drive/My Drive/JantaHack/test_BdBKkAj_L87Nc3S.csv')
ss = pd.read_csv('/content/drive/My Drive/JantaHack/sample_submission_KVKNmI7.csv')

In [0]:
train.head()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041


# Prophet Model

In [0]:
final_test_df = pd.DataFrame(columns = ss.columns)
for junc in range(1,5):
  print("Junction : ",junc)
  df = train[train['Junction'] == junc][['DateTime','Vehicles','ID']]
  df['DateTime'] = pd.to_datetime(df['DateTime'])
  prophet_data = df[['DateTime','Vehicles']]
  prophet_data.columns = ['ds','y']

  m = Prophet()
  m.add_country_holidays(country_name='IN')
  m.fit(prophet_data)

  df_test = test[test['Junction'] == junc]
  df_test['DateTime'] = pd.to_datetime(df_test['DateTime'])
  prophet_test = df_test[['DateTime']]
  prophet_test.columns = ['ds']

  forecast = m.predict(prophet_test)
  test_hat = forecast['yhat'].values.tolist()
  test_high = forecast['yhat_upper'].values.tolist()
  test_low = forecast['yhat_lower'].values.tolist()

  test_pred_df = pd.DataFrame(columns = ss.columns)
  test_pred_df['ID'] = df_test['ID']
  test_pred_df['Vehicles'] = test_hat
  if junc == 1:
    final_test_df = test_pred_df
  else:
    final_test_df = pd.concat([final_test_df,test_pred_df])


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Junction :  1


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Junction :  2


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Junction :  3


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Junction :  4


In [0]:
final_test_df.to_csv('Submission_4.csv',index=False)

In [0]:
final_test_df.ID.unique().shape

(11808,)

# Pycaret Model

In [0]:
test.head()

,DateTime,Junction,ID
0,2017-07-01 00:00:00,1,20170701001
1,2017-07-01 01:00:00,1,20170701011
2,2017-07-01 02:00:00,1,20170701021
3,2017-07-01 03:00:00,1,20170701031
4,2017-07-01 04:00:00,1,20170701041
